In [1]:
import pandas as pd
import os
import requests
import json

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [2]:
#Setting the api_key variable
api_key = os.environ["FOURSQUARE_API_KEY"]

In [3]:
#Reading in the vancouver_bikes dataframe, which includes the locations of all bike stations in Vancouver
vancouver_bikes = pd.read_csv("vancouver_bikes.csv")
vancouver_bikes.head()

,uid,latitude,longitude,free_bikes,empty_slots,station_name
0,1,49.262487,-123.114397,1,32,10th & Cambie
1,4,49.274566,-123.121817,10,5,Yaletown-Roundhouse Station
2,5,49.279764,-123.110154,8,16,Dunsmuir & Beatty
3,7,49.260599,-123.113504,3,12,12th & Yukon (City Hall)
4,8,49.264215,-123.117772,3,12,8th & Ash


In [6]:
#Creating a function to make an API request to Foursquare
def get_venues_fsq(latitude, longitude, radius, api_key, categories):
    url = "https://api.foursquare.com/v3/places/search"
    
    headers = {
        "Accept": "application/json",
        "Authorization": api_key
    }
    
    params = {
        "ll": f"{latitude},{longitude}",
        "radius": radius,
        "categories": categories,
        "limit": 50
    }
    
    response = requests.get(url, headers=headers, params=params)
    return response

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [7]:
#Creating an empty list for number of parks within 1000 radius from each bike station
number_of_parks_list = []

# Creating an empty list to store the JSON responses for each location
foursquare_data = []
        
for i, r in vancouver_bikes.iterrows():
    lat = r["latitude"]
    long = r["longitude"]
#Making a request for parks as a venue category within a 1000-meter radius from a bike station
    res = get_venues_fsq(lat, long, radius = 1000, api_key = api_key, categories = 16032)
    foursquare_results = res.json()
#Checking the number of parks for each given location using the len function
    number_of_parks = len(foursquare_results["results"])
#Appending the number of parks for each bike location to a list
    parks_list.append(number_of_parks)
    
    for row in foursquare_results['results']:
        foursquare_data.append({
            'fsq_id'           :row['fsq_id'],
            'distance_station' :row['distance'],
            'latitude'         :row['geocodes']['main']['latitude'],
            'longitude'        :row['geocodes']['main']['longitude'],
            'name'             :row['name'],
            'category_name'    :row['categories'][0]['name'],
            'uid'              :r['uid']
            }) 

Put your parsed results into a DataFrame

In [ ]:
parks_fsq = pd.DataFrame(foursquare_data)

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Get the top 10 restaurants according to their rating